<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Kaiven Yang Su, 20021460' #@param {type:"string"}






In [ ]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

In [ ]:
import pandas as pd
import numpy as np

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [249]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df

id                                               name    host_id  \
0       20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1       25235                  Large 2 Bedroom Great for Groups!      87773   
2       76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
3      135465                Garden apartment close to Manhattan     663879   
4      169002                   Modern Space in Charming Pre-war     805344   
..        ...                                                ...        ...   
799  36413779     Sunny & spacious room in a 3 Br - Williamsburg    2449230   
800  36421287  Nice & Lovely room in LIC.5 stops to Times Square  203866020   
801  36439512                       The Bushwick Backhouse Loft.  180336853   
802  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
803  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3    Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4           Alec           Manhattan              Harlem  40.82411  -73.94934   
..           ...                 ...                 ...       ...        ...   
799       Itamar            Brooklyn        Williamsburg  40.71812  -73.95495   
800        Karma              Queens           Sunnyside  40.73799  -73.92749   
801         Dash            Brooklyn            Bushwick  40.70470  -73.92250   
802      Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
803       Brenda           Manhattan              Inwood  40.86845  -73.92449   

           room_type  price  minimum_nights  number_of_reviews last_review  \
0    Entire home/apt    100               5                168  2018-07-22   
1    Entire home/apt    125              90                162  2019-06-28   
2    Entire home/apt    169               2                398  2019-06-28   
3    Entire home/apt    170               3                 86  2019-06-20   
4       Private room     65               2                 41  2019-06-16   
..               ...    ...             ...                ...         ...   
799     Private room     80               5                  0         NaN   
800     Private room     60               1                  0         NaN   
801     Private room     70               2                  0         NaN   
802     Private room    800               1                  0         NaN   
803     Private room     80               1                  0         NaN   

     reviews_per_month  calculated_host_listings_count  availability_365  
0                 1.57                               1                 0  
1                 1.46                               2               137  
2                 3.97                               3               182  
3                 0.91                               2               286  
4                 0.43                               2                59  
..                 ...                             ...               ...  
799                NaN                               1                38  
800                NaN                               1                34  
801                NaN                               1               255  
802                NaN                               1                23  
803                NaN                               1                79  

[804 rows x 16 columns]

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
df.groupby('host_name')['calculated_host_listings_count'].nlargest(3).sort_values(ascending=False).head(6)

host_name        
Sonder (NYC)  678    327
              677    327
              676    327
Blueground    618    232
              665    232
Kara          242    121
Name: calculated_host_listings_count, dtype: int64

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [ ]:
preco_mais_de_um_imovel = df[df['calculated_host_listings_count'] > 1]['price'].mean()
preco_mais_um_imovel = df[df['calculated_host_listings_count'] == 1]['price'].mean()
print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD -24.28


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
#percentage of outlier prices
df_Manhattan = df[df['neighbourhood_group'] == 'Manhattan']
def identify_outliers(data):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outliers = data[(data < lower_bound) | (data > upper_bound)].index.tolist()
  return outliers

outliers = identify_outliers(df_Manhattan['price'])
num_outliers = len(outliers)
perc_Manhattan = (num_outliers / len(df_Manhattan)) * 100
print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')

Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [ ]:
df_with_taxes = pd.merge(df, df_taxes, on='id', how='left')
df_with_taxes
df_with_taxes = df_with_taxes.fillna(0)
df_with_taxes

id                                               name    host_id  \
0        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
2        25235                  Large 2 Bedroom Great for Groups!      87773   
3        25235                  Large 2 Bedroom Great for Groups!      87773   
4        76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
...        ...                                                ...        ...   
1563  36439512                       The Bushwick Backhouse Loft.  180336853   
1564  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1565  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1566  36472171                      1 bedroom in sunlit apartment   99144947   
1567  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
2         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
...          ...                 ...                 ...       ...        ...   
1563        Dash            Brooklyn            Bushwick  40.70470  -73.92250   
1564     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1565     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1566      Brenda           Manhattan              Inwood  40.86845  -73.92449   
1567      Brenda           Manhattan              Inwood  40.86845  -73.92449   

            room_type  price  minimum_nights  number_of_reviews last_review  \
0     Entire home/apt    100               5                168  2018-07-22   
1     Entire home/apt    100               5                168  2018-07-22   
2     Entire home/apt    125              90                162  2019-06-28   
3     Entire home/apt    125              90                162  2019-06-28   
4     Entire home/apt    169               2                398  2019-06-28   
...               ...    ...             ...                ...         ...   
1563     Private room     70               2                  0           0   
1564     Private room    800               1                  0           0   
1565     Private room    800               1                  0           0   
1566     Private room     80               1                  0           0   
1567     Private room     80               1                  0           0   

      reviews_per_month  calculated_host_listings_count  availability_365  \
0                  1.57                               1                 0   
1                  1.57                               1                 0   
2                  1.46                               2               137   
3                  1.46                               2               137   
4                  3.97                               3               182   
...                 ...                             ...               ...   
1563               0.00                               1               255   
1564               0.00                               1                23   
1565               0.00                               1                23   
1566               0.00                               1                79   
1567               0.00                               1                79   

       type   value  
0     taxes   789.0  
1     fines   796.0  
2     taxes  1779.0  
3     fines     0.0  
4     taxes  1516.0  
...     ...     ...  
1563  fines     0.0  
1564  taxes  1028.0  
1565  

In [ ]:
print('tax_mean_by_neighbourhood')
df_with_taxes[df_with_taxes['type']=='taxes'].groupby('neighbourhood_group')[['value']].mean().astype(int)

tax_mean_by_neighbourhood


value
neighbourhood_group       
Bronx                  851
Brooklyn               858
Manhattan              888
Queens                 804
Staten Island         1157

In [ ]:
print('fine_mean_by_neighbourhood')
df_with_taxes[df_with_taxes['type'] == 'fines'].groupby('neighbourhood_group')[['value']].mean().astype(int)

fine_mean_by_neighbourhood


value
neighbourhood_group       
Bronx                  126
Brooklyn               314
Manhattan              314
Queens                 322
Staten Island          581

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
df_report = df_with_taxes.sort_values(by=['neighbourhood_group', 'neighbourhood'])
df_neighbourhood_group_mean = df_report.groupby(['neighbourhood_group'])['price'].mean().astype(int).reset_index()
df_neighbourhood_group_mean['neighbourhood'] = 'mean'
df_report = pd.concat([df_neighbourhood_group_mean, df_report])
df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])
df_report

neighbourhood_group  price      neighbourhood          id  \
862                Bronx     29            Belmont  21314250.0   
863                Bronx     29            Belmont  21314250.0   
1498               Bronx     67  Claremont Village  35524274.0   
1499               Bronx     67  Claremont Village  35524274.0   
689                Bronx     40          Concourse  17242854.0   
...                  ...    ...                ...         ...   
1090       Staten Island    130         St. George  26889078.0   
1389       Staten Island    135          Todt Hill  33914554.0   
1390       Staten Island    135          Todt Hill  33914554.0   
796        Staten Island     59      Tompkinsville  20034244.0   
4          Staten Island     91               mean         NaN   

                                                   name      host_id  \
862                                    5 STAR COZY ROOM  154238443.0   
863                                    5 STAR COZY ROOM  154238443.0   
1498                                  Master Room 315-B  267272502.0   
1499                                  Master Room 315-B  267272502.0   
689                   Yankee Stadium apartment in Bronx    8159536.0   
...                                                 ...          ...   
1090          Staten Island Garden Apartment Near Ferry  199535935.0   
1389         3 Bedroom Full house - 15mins to NYC/Ferry  161357125.0   
1390         3 Bedroom Full house - 15mins to NYC/Ferry  161357125.0   
796   Two rooms in one. A family friendly neighborhood.  113295877.0   
4                                                   NaN          NaN   

     host_name  latitude  longitude        room_type  minimum_nights  \
862        Mrs  40.85579  -73.88512     Private room             3.0   
863        Mrs  40.85579  -73.88512     Private room             3.0   
1498  Orfelina  40.84308  -73.90652     Private room             3.0   
1499  Orfelina  40.84308  -73.90652     Private room             3.0   
689      Genae  40.83243  -73.92035     Private room             1.0   
...        ...       ...        ...              ...             ...   
1090       Tim  40.64641  -74.08502  Entire home/apt             2.0   
1389    Mohsin  40.61041  -74.11593  Entire home/apt             3.0   
1390    Mohsin  40.61041  -74.11593  Entire home/apt             3.0   
796   Jonathan  40.63529  -74.09068     Private room             3.0   
4          NaN       NaN        NaN              NaN             NaN   

      number_of_reviews last_review  reviews_per_month  \
862                 9.0  2019-06-12               0.43   
863                 9.0  2019-06-12               0.43   
1498                0.0           0               0.00   
1499                0.0           0               0.00   
689                11.0  2019-02-20               0.42   
...                 ...         ...                ...   
1090               52.0  2019-06-24               4.60   
1389                2.0  2019-06-23               0.86   
1390                2.0  2019-06-23               0.86   
796                26.0  2019-06-22               1.11   
4                   NaN         NaN                NaN   

      calculated_host_listings_count  availability_365   type   value  
862                              2.0             322.0  taxes  1787.0  
863                              2.0             322.0  fines     0.0  
1498                             2.0             113.0  taxes   948.0  
1499                             2.0             113.0  fines     0.0  
689                              3.0             239.0  taxes  1764.0  
...                              ...               ...    ...     ...  
1090                             1.0              96.0  fines     0.0  
1389                             3.0              36.0  taxes  1498.0  
1390                             3.0              36.0  fines     0.0  
796                              3.0             351.0      0    

In [ ]:
df_report['line_id'] = range(1, len(df_report) + 1)
df_report['page_id'] = (df_report['line_id'] + 29) // 30

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 18615


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group       price  neighbourhood          id  \
964        Staten Island   20.000000    South Beach  22977021.0   
965        Staten Island   20.000000    South Beach  22977021.0   
1183       Staten Island  100.000000    South Beach  29393457.0   
1184       Staten Island  100.000000    South Beach  29393457.0   
1089       Staten Island  130.000000     St. George  26889078.0   
1090       Staten Island  130.000000     St. George  26889078.0   
1389       Staten Island  135.000000      Todt Hill  33914554.0   
1390       Staten Island  135.000000      Todt Hill  33914554.0   
796        Staten Island   59.000000  Tompkinsville  20034244.0   
4          Staten Island   91.769231           mean         NaN   

                                                   name      host_id  \
964   Spacious Room4 Rent Near Hospital in Staten Is...   46723570.0   
965   Spacious Room4 Rent Near Hospital in Staten Is...   46723570.0   
1183           2 Private Rooms Near Staten Island Ferry  204823078.0   
1184           2 Private Rooms Near Staten Island Ferry  204823078.0   
1089          Staten Island Garden Apartment Near Ferry  199535935.0   
1090          Staten Island Garden Apartment Near Ferry  199535935.0   
1389         3 Bedroom Full house - 15mins to NYC/Ferry  161357125.0   
1390         3 Bedroom Full house - 15mins to NYC/Ferry  161357125.0   
796   Two rooms in one. A family friendly neighborhood.  113295877.0   
4                                                   NaN          NaN   

      host_name  latitude  longitude        room_type  minimum_nights  \
964   Kartikeya  40.58950  -74.09663      Shared room             4.0   
965   Kartikeya  40.58950  -74.09663      Shared room             4.0   
1183      Maria  40.58639  -74.08943     Private room             2.0   
1184      Maria  40.58639  -74.08943     Private room             2.0   
1089        Tim  40.64641  -74.08502  Entire home/apt             2.0   
1090        Tim  40.64641  -74.08502  Entire home/apt             2.0   
1389     Mohsin  40.61041  -74.11593  Entire home/apt             3.0   
1390     Mohsin  40.61041  -74.11593  Entire home/apt             3.0   
796    Jonathan  40.63529  -74.09068     Private room             3.0   
4           NaN       NaN        NaN              NaN             NaN   

      number_of_reviews last_review  reviews_per_month  \
964                 1.0  2018-02-02               0.06   
965                 1.0  2018-02-02               0.06   
1183                6.0  2019-05-26               0.73   
1184                6.0  2019-05-26               0.73   
1089               52.0  2019-06-24               4.60   
1090               52.0  2019-06-24               4.60   
1389                2.0  2019-06-23               0.86   
1390                2.0  2019-06-23               0.86   
796                26.0  2019-06-22               1.11   
4                   NaN         NaN                NaN   

      calculated_host_listings_count  availability_365   type   value  \
964                              1.0               0.0  taxes  1432.0   
965                              1.0               0.0  fines  1813.0   
1183                             2.0              86.0  taxes  1360.0   
1184                             2.0              86.0  fines     0.0   
1089                             1.0              96.0  taxes  1604.0   
1090                             1.0              96.0  fines     0.0   
1389                             3.0              36.0  taxes  1498.0   
1390                             3.0              36.0  fines     0.0   
796                              3.0             351.0      0     0.0   
4                                NaN               NaN    NaN     NaN   

      line_id  page_id  
964      1564       53  
965      1565       53  
1183     1566       53  
1184     1567       53  
1089     1568       53  
1090     1569       53  
1389     1570       53  
1390     1571       53  
79